June 7 through June 17
Future steps - Must do:
* Using the other data files (Feature Engineering, Create flat dataset)
* Data visualization
* Normalizing data
* Handling skewed data
Report:
* Preliminary results

June 18 through July 1
Report:
* WEEK1: I should have a skeleton of all the areas of the report to share with mentor.
* WEEK2: I should have a first full version of the report to share with mentor and ready to submit
Future steps - Nice to do:
* CREATE A scikit-learn Pipeline
* MAKE AN ENSEMBLE METHOD OF MULTIPLE MODELS
* Measuring model/learner performance using other evaluation metric (LR+, f1, r2)?
* Hyperparameter tunening (different values until I find a max, use higher percentage of training & validation data)
* Review Handling missing or invalid data, other better methods?

DONE:
* Handling missing or invalid data
* Split train & validation set
* Hyperparameter tunening
* Measuring model/learner performance using evaluation metric (auc_roc_score)

Preliminary results:
- Categorical values turned into numerical features with one-hot encoding scheme
- Fill missing or wrong values
- RFR Default values, only application training data, score: 0.591
- RFR Best values with GridSearchCV, only application training data, score: 0.722 <- 0.62
- RFR Best values with GridSearchCV, ALL available data, score: ?
- ???? Best values with GridSearchCV, ALL available data, score: ?
- Normalizing Data
- Unskewing Data

Abbreviations:
RFR: Random Forest Regressor
????: Another regressor

# **Sections:**
[1. Import libraries & support functions](#1)  
[2. Dataset preparation](#2)  
[3. Exploratory Data Analysis (EDA)](#3)  
&nbsp; [3.1 Application Train](#3.1)  
[4. Feature Engineering](#4)  
&nbsp; [4.1 Handle Skewed Continuous Data](#4.1)  
&nbsp; [4.2 Normalize Continuous Data](#4.2)  
&nbsp; [4.3 Handle Categorical Variables](#4.3)  
&nbsp; [4.4 Handle Missing Data](#4.4)  
&nbsp; [4.5 Create a Flat Dataset](#4.5)  
[5. Split Data into Training and Validation](#5)  
[6. Hyperparameter Tuning](#6)  
[7. Model Fitting & Prediction](#7)  

In [58]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSVfile I/O (e.g. pd.read_csv)
import os
from plotly.offline import init_notebook_mode, iplot
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly import tools
# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
from sklearn.ensemble import RandomForestRegressor
# Add evaluation metric to measure the model's performance
# Regression metrics available:
# http://scikit-learn.org/stable/modules/classes.html#regression-metrics
# http://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics
# http://scikit-learn.org/stable/modules/model_evaluation.html#receiver-operating-characteristic-roc
# http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
# http://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html
# Cannot use sklearn.metrics.accuracy_score as it is a Classification metric
from sklearn.metrics import make_scorer, r2_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from time import time


'''
def bar_hor(df, col, title, color, w=None, h=None, lm=0, limit=100, return_trace=False, rev=False, xlb = False):
    cnt_srs = df[col].value_counts()
    yy = cnt_srs.head(limit).index[::-1] 
    xx = cnt_srs.head(limit).values[::-1] 
    if rev:
        yy = cnt_srs.tail(limit).index[::-1] 
        xx = cnt_srs.tail(limit).values[::-1] 
    if xlb:
        trace = go.Bar(y=xlb, x=xx, orientation = 'h', marker=dict(color=color))
    else:
        trace = go.Bar(y=yy, x=xx, orientation = 'h', marker=dict(color=color))
    if return_trace:
        return trace 
    layout = dict(title=title, margin=dict(l=lm), width=w, height=h)
    data = [trace]
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

def gp(col, title):
    df1 = data_train[data_train["TARGET"] == 1]
    df0 = data_train[data_train["TARGET"] == 0]
    a1 = df1[col].value_counts()
    b1 = df0[col].value_counts()
    
    total = dict(data_train[col].value_counts())
    x0 = a1.index
    x1 = b1.index
    
    y0 = [float(x)*100 / total[x0[i]] for i,x in enumerate(a1.values)]
    y1 = [float(x)*100 / total[x1[i]] for i,x in enumerate(b1.values)]

    trace1 = go.Bar(x=a1.index, y=y0, name='Target : 1', marker=dict(color="#96D38C"))
    trace2 = go.Bar(x=b1.index, y=y1, name='Target : 0', marker=dict(color="#FEBFB3"))
    return trace1, trace2 
'''

print(os.listdir("../input"))

In [59]:
from IPython.display import display # Allows the use of display() for DataFrames

# Data exploration

# Load the application training dataset application_train.csv
data_train = pd.read_csv("../input/application_train.csv")

# Load the application testing dataset application_test.csv
data_test = pd.read_csv("../input/application_test.csv")

'''
# Display the first 5 records of application_train.csv
display(data_train.head())

# Display the first 5 records of application_test.csv
display(data_test.head())
'''

# DataFrame statistics summary for selected columns
data_train[["AMT_INCOME_TOTAL", "AMT_CREDIT", "AMT_ANNUITY", "AMT_GOODS_PRICE"]].describe()
#print(len(data_train.columns))

In [60]:
# Total number of records
print("Total number of records in the train dataset: {:,}".format(len(data_train)))
# Total number of features. Excluding the load ids (SK_ID_CURR) and the target variable (TARGET).
print("Total number of features in the train dataset: {}".format(data_train.shape[1] - 2))

# Total number of records
print("Total number of records in the test dataset: {:,}".format(len(data_test)))
# Total number of features. Excluding the load ids (SK_ID_CURR). There is NO target variable (TARGET) in the test dataset.
print("Total number of features in the test dataset: {}".format(data_test.shape[1] - 1))

In [61]:
# GRAPHS ARE NOT DISPLAYING
'''
# Target Variable Distribution 
bar_hor(data_train, "TARGET", "Distribution of Target Variable" , ["#44ff54", '#ff4444'], h=350, w=600, lm=200, xlb = ['Target : 1','Target : 0'])

tr0 = bar_hor(data_train, "CODE_GENDER", "Distribution of CODE_GENDER Variable" ,"#f975ae", w=700, lm=100, return_trace= True)
tr1, tr2 = gp('CODE_GENDER', 'Distribution of Target with Applicant Gender')

fig = tools.make_subplots(rows=1, cols=3, print_grid=False, subplot_titles = ["Gender Distribution" , "Gender, Target=1" ,"Gender, Target=0"])
fig.append_trace(tr0, 1, 1);
fig.append_trace(tr1, 1, 2);
fig.append_trace(tr2, 1, 3);
fig['layout'].update(height=350, showlegend=False, margin=dict(l=50));
iplot(fig);
'''

In [62]:
# Transforming skewed continuous features

In [63]:
# Normalizing numerical features
from sklearn.preprocessing import MinMaxScaler

In [64]:
# Turn categorical variables into numerical features using the one-hot encoding scheme

# Total number of features. Excluding the load ids (SK_ID_CURR) and the target variable (TARGET)
print("Total number of features of train data, before one-hot encoding: {}".format(data_train.shape[1] - 2))
# Total number of features. Excluding the load ids (SK_ID_CURR). The test dataset has NO target label (TARGET)
print("Total number of features of test data, before one-hot encoding: {}".format(data_test.shape[1] - 1))

# One-hot encoding
data_train_encoded = pd.get_dummies(data_train)
data_test_encoded = pd.get_dummies(data_test)

# Total number of features. Excluding the load ids (SK_ID_CURR) and the target variable (TARGET)
print("Total number of features of train data, after one-hot encoding: {}".format(data_train_encoded.shape[1] - 2))
# Total number of features. Excluding the load ids (SK_ID_CURR). The test dataset has NO target label (TARGET)
print("Total number of features of test data, after one-hot encoding: {}".format(data_test_encoded.shape[1] - 1))

# New list of features
#print(list(data_train_encoded.columns))

# Determine what columns are missing
train_list = list(data_train_encoded.columns)
test_list = list(data_test_encoded.columns)
difference = [e for e in train_list if e not in test_list]
print(difference)

# Add those columns to the test set will all zeros
data_test_encoded_complete = data_test_encoded
for e in difference:
    if e != 'TARGET':
        data_test_encoded_complete[e] = 0

print("Total number of features of test data, after one-hot encoding: {}".format(data_test_encoded_complete.shape[1] - 1))

In [65]:
# Separate the target label from the train dataset. The column we are interested is 'TARGET'. Name it target_train.
target_train = data_train['TARGET']
print("Training target label has {:,} samples".format(target_train.shape[0]))

# Remove target label from the train dataset and rename to features_train.
features_train = data_train_encoded.drop(['TARGET'], axis=1)

# Test data has no taget label 'TARGET' in the dataset
features_test = data_test_encoded_complete

In [66]:
# Handle missing values
# Source: https://www.kaggle.com/dansbecker/handling-missing-values
# http://scikit-learn.org/dev/modules/generated/sklearn.impute.SimpleImputer.html

from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
features_train = my_imputer.fit_transform(features_train)
features_test = my_imputer.fit_transform(features_test)

In [67]:
# Shuffle and split the data
# http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
from sklearn.model_selection import train_test_split

# Split the 'features' and 'target label' data into training and validating sets
X_train, X_val, y_train, y_val = train_test_split(features_train,
                                                  target_train,
                                                  test_size=0.2,
                                                  random_state=42)

print("Original Training set has {:,} samples.".format(features_train.shape[0]))
print("After split Training set has {:,} samples.".format(X_train.shape[0]))
print("After split Validating set has {:,} samples.".format(X_val.shape[0]))
print("Testing set has {:,} samples.".format(data_test.shape[0]))

In [68]:
# Run GridSearchCV or fully train an estimator
run_mode = 'grid_search' # 'grid_search' or 'train_estimator'

In [72]:
# Run GridSearchCV
if run_mode == 'grid_search':
    perc_samples = 0.001
    print("Preparing to run Hyperparameters tunning with GridSearchCV using {0:.2f}% of the training samples".format(perc_samples * 100))
    features_train_small = X_train[:int(perc_samples * X_train.shape[0])]
    target_train_small = y_train[:int(perc_samples * y_train.shape[0])]
    features_val_small = X_val[:int(perc_samples * X_val.shape[0])]
    target_val_small = y_val[:int(perc_samples * y_val.shape[0])]
    #features_test_small = features_test[:int(perc_samples * features_test.shape[0])]

    # Initialize the Estimator (Learner or Regression Model)
    estimator = RandomForestRegressor(n_jobs=-1,
                                      random_state=42,
                                      verbose=0)

    # Determine which Parameters to tune
    '''
    Tested so far:
    parameters = {
        'n_estimators': [9, 10, 11, 12, 13, 14, 15],
        'criterion': ['mse', 'mae'],
        'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7],
        'max_features': [0.01, 0.1, 0.25, 0.45, 0.5, 0.55, 0.6, 0.75],
        'min_samples_split': [2, 3, 4, 5],
        'warm_start': [False, True]
    }
    '''
    parameters = {
        'n_estimators': [115, 125, 135],
        'min_samples_leaf': [62, 75, 87],
        'max_features': [0.18, 0.2, 0.23],
        'min_samples_split': [2, 3],
    }

    # Create a scorer to measure hyperparameters performance
    scorer = make_scorer(roc_auc_score)

    # Create GridSearchCV grid object
    grid_obj = GridSearchCV(estimator=estimator, 
                            param_grid=parameters, 
                            scoring=scorer)

    # Fit the GridSearchCV grid object with the reduced training dataset and find the best hyperparameters
    start = time()
    grid_fit = grid_obj.fit(features_train_small, target_train_small)
    end = time()
    grid_fit_time = (end - start) / 60 # Ellapsed time in minutes
    print("GridSearchCV estimator fit time: {0:.2f} minutes".format((end - start) / 60))

    # Get the best estimator
    best_est = grid_obj.best_estimator_
    print("Best Estimator: \n{}\n".format(best_est))

    # Get the best score
    best_score = grid_obj.best_score_
    print("Best Estimator Score: {}\n".format(best_score))

    # Get the best parameters
    best_params = grid_obj.best_params_
    print("Best Hyperparameters that yield the best score: \n{}\n".format(best_params))

    # Make predictions with unoptimized estimator on the validation set
    pred_val = (estimator.fit(features_train_small, target_train_small)).predict(features_val_small)
    print("Unoptimized Estimator prediction score on Validation set: \t{}".format(roc_auc_score(target_val_small, pred_val)))

    # Predict with the best estimator on the validation set
    best_pred_val = best_est.predict(features_val_small)
    print("Optimized Estimator prediction score on Validation set: \t{}".format(roc_auc_score(target_val_small, best_pred_val)))

    # Predict with the best estimator on the testing set
    #pred_test = best_est.predict(features_test)

In [ ]:
# Train estimator
# TODO: rename or remove '_small', it might be '_full' or nothing
if run_mode == 'train_estimator':
    # Use the full training and validation datasets to fit the estimator with the best hyperparameters
    perc_samples = 1
    print("Preparing to train an estimator using {0:.2f}% of the training dataset".format(perc_samples * 100))
    features_train_small = X_train[:int(perc_samples * X_train.shape[0])]
    target_train_small = y_train[:int(perc_samples * y_train.shape[0])]
    features_val_small = X_val[:int(perc_samples * X_val.shape[0])]
    target_val_small = y_val[:int(perc_samples * y_val.shape[0])]
    features_test_small = features_test[:int(perc_samples * features_test.shape[0])]

    # Initialize the Estimator (Learner or Regression Model) with the best hyperparameters
    '''
    estimator = RandomForestRegressor(criterion='mae', # default='mse', VERY SLOW
                                      min_samples_split=2, # default=2
                                      warm_start=False) # default=False
    Best Performance:
    estimator = RandomForestRegressor(n_estimators=125, # default=10
                                      max_features=0.2, # default='auto'
                                      min_samples_split=2, # default=2
                                      min_samples_leaf=75, # default=1
                                      n_jobs=-1, # default=1
                                      random_state=42, # default=None
                                      verbose=2) # default=0
                                      ROC_AUC_SCORE:  0.7417496532130599 (% samples: 0.15)
                                      LEADERBOARD SCORE: 0.722 (NO improvement)

    estimator = RandomForestRegressor(n_estimators=100, # default=10
                                      max_features=0.2, # default='auto'
                                      min_samples_split=3, # default=2
                                      min_samples_leaf=50, # default=1
                                      n_jobs=-1, # default=1
                                      random_state=42, # default=None
                                      verbose=2) # default=0
                                      ROC_AUC_SCORE:  0.7286786065442892 (% samples: 0.1)
                                      LEADERBOARD SCORE: 0.722

    estimator = RandomForestRegressor(n_estimators=12, # default=10
                                      max_features=0.45, # default='auto'
                                      min_samples_leaf=2, # default=1
                                      n_jobs=-1, # default=1
                                      random_state=42, # default=None
                                      verbose=2) # default=0
                                      LEADERBOARD SCORE: 0.62
    '''
    estimator = RandomForestRegressor(n_estimators=125, # default=10
                                      max_features=0.2, # default='auto'
                                      min_samples_split=2, # default=2
                                      min_samples_leaf=75, # default=1
                                      n_jobs=-1, # default=1
                                      random_state=42, # default=None
                                      verbose=0) # default=0

    # Fit the estimator with the training dataset
    start = time()
    estimator.fit(features_train_small, target_train_small)
    end = time()
    print("Estimator fit time: {0:.2f} minutes".format((end - start) / 60))

    # Predict with the validation dataset
    pred_val = estimator.predict(features_val_small)
    print("Estimator prediction score on Validation set: \t{}".format(roc_auc_score(target_val_small, pred_val)))
    
    # Predict using the 'test' dataset for submission
    pred_test = estimator.predict(features_test_small)
    #pred_test = estimator.predict(features_test)

    # Prepare prediction for submission
    submission = pd.DataFrame()
    # Need to replace data_test_encoded_complete
    submission['SK_ID_CURR'] = data_test_encoded_complete['SK_ID_CURR'][:int(perc_samples * features_test.shape[0])]
    submission['TARGET'] = pred_test
    submission.head()
    submission.to_csv('RFR.csv', index=False)